# Preview Quiz prompt

In [11]:
from pydantic import BaseModel
from openai import OpenAI
from openai.lib._parsing import type_to_response_format_param
import json
from getpass import getpass

openai_api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [12]:
from jinja2 import Template

prompt_template = Template("""
[PURPOSE]
Preview lesson content through quizzes to give a sneak peek before the actual lesson.
The quizzes must ask key expressions/the most structurally important words themselves written in or based on the GOAL/DESCRIPTION.

[QUIZ GUIDELINES]
- Create two preview quizzes per topic. One from KEY_EXPRESSION, the other from SUB_EXPRESSIONS.
- Put blanks on the KEY_EXPRESSION and SUB_EXPRESSION itself, no elsewhere.
- No blanks or tildes in Korean translation
- No tildes in the quiz.
- Make sure you have NO multiple "equally valid" answers
- Blanks must directly ask the key expressions/the most meaningful words themselves based on the DESCRIPTION.
- Answer index = the position number of the answer.
- Ensure that the quiz is a verb, adjective, or a key phrase from the lesson.
- No proper nouns can be tested.
- Create SAMPLE SCRIPT for each topic at the end of each topic.


[POLICY]
Each quiz needs:
- A blank (test key patterns itself)
- Complete Korean translation (no blanks or tildes)
- Three options (one correct, two wrong)
- No multiple "equally valid" answers/options
- Wrong options must be distinctly wrong.
- Shuffle the order of answer choices='answer indexes (1-3)' to avoid predictability.
- SAMPLE SCRIPT must not be the same as the quizzes. Change one or two words to create a dialogue of different topic, but maintain the structure.

[Example]
```
Input: 
    {
    "DESCRIPTION": "This is my friend, ~와 같은 표현을 사용해 사람을 소개하기.",
    "KEY_EXPRESSION": "A: This is my friend, Jake.",
    "SUB_EXPRESSION": [
      "A: Let me introduce my friend, ~. A: Meet my friend, ~."
    ]
  }

Output:
    {
    "goal": [
      "다른 사람을 소개하는 표현을 사용할 수 있어요.",
      "This is my friend, ~과 같은 표현을 사용하여 친구를 소개할 수 있어요."
    ],
    "preview_quizzes": [
      {
        "fill_in_blank_quiz": "___ is my friend, Jennie.",
        "korean_translation": "이쪽은 내 친구 Jennie야.",
        "options": ["I", "They", "This"],
        "answer": "This",
        "answer_index": 3
      },
      {
        "fill_in_blank_quiz": "Let me ___ my friend, Sarah.",
        "korean_translation": "내 친구 Sarah를 소개할게.",
        "options": ["introduce", "inter", "intro"],
        "answer": "introduce",
        "answer_index": 2
      }
    ]
    "sample_script" : [
      {
        "sample_script_eng" : "A: This is my friend, Emma. B: Nice to meet you, Emma! A: Let me introduce my friend, David. B: Hey, David! Glad to meet you.",
        "sample_script_kor" : "A: 얘는 내 친구, 엠마야! B: 반가워, 엠마! A: 내 친구 데이빗을 소개할게. B: 안녕, 데이빗! 만나서 반가워."
      }                       
    ]
  }
}
```

[Input]
```
DESCRIPTION: {{description}}
KEY EXPRESSION: {{key_expression}}
SUB EXPRESSION: {{sub_expression}}
```

""")

In [13]:
prompt = prompt_template.render(
    description = "자기소개와 출신지에 대해 말하고, 이에 반응하기.",
    key_expression = "A: My name is Felix. I’m from Australia. B: Glad to meet you.",
    sub_expression = [
        "A: Hi, I’m (). I come from (). B: Nice to meet you. A: Hello, my name is (). I’m from (). B: Great to meet you."
    ]
)

In [14]:
print(prompt)


[PURPOSE]
Preview lesson content through quizzes to give a sneak peek before the actual lesson.
The quizzes must ask key expressions/the most structurally important words themselves written in or based on the GOAL/DESCRIPTION.

[QUIZ GUIDELINES]
- Create two preview quizzes per topic. One from KEY_EXPRESSION, the other from SUB_EXPRESSIONS.
- Put blanks on the KEY_EXPRESSION and SUB_EXPRESSION itself, no elsewhere.
- No blanks or tildes in Korean translation
- No tildes in the quiz.
- Make sure you have NO multiple "equally valid" answers
- Blanks must directly ask the key expressions/the most meaningful words themselves based on the DESCRIPTION.
- Answer index = the position number of the answer.
- Ensure that the quiz is a verb, adjective, or a key phrase from the lesson.
- No proper nouns can be tested.
- Create SAMPLE SCRIPT for each topic at the end of each topic.


[POLICY]
Each quiz needs:
- A blank (test key patterns itself)
- Complete Korean translation (no blanks or tildes)


In [36]:
class PreviewQuiz(BaseModel):
    fill_in_blank_quiz : str
    korean_translation : str
    options : list[str]
    answer : str
    answer_index : int

class SampleScript(BaseModel):
    sample_script_eng : str
    sample_script_kor : str

class QuizResponse(BaseModel):
    goal : list[str]
    preview_quizzes : list[PreviewQuiz]
    sample_script : list[SampleScript]

In [37]:
response_format = type_to_response_format_param(QuizResponse)

In [17]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'$defs': {'PreviewQuiz': {'properties': {'fill_in_blank_quiz': {'title': 'Fill In Blank Quiz',
       'type': 'string'},
      'korean_translation': {'title': 'Korean Translation', 'type': 'string'},
      'options': {'items': {'type': 'string'},
       'title': 'Options',
       'type': 'array'},
      'answer': {'title': 'Answer', 'type': 'string'},
      'answer_index': {'title': 'Answer Index', 'type': 'integer'}},
     'required': ['fill_in_blank_quiz',
      'korean_translation',
      'options',
      'answer',
      'answer_index'],
     'title': 'PreviewQuiz',
     'type': 'object',
     'additionalProperties': False},
    'SampleScript': {'properties': {'sample_script_eng': {'title': 'Sample Script Eng',
       'type': 'string'},
      'sample_script_kor': {'title': 'Sample Script Kor', 'type': 'string'}},
     'required': ['sample_script_eng', 'sample_script_kor'],
     'title': 'SampleScript',
     'type': 'object',
     '

In [38]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You will provide quizzes related to the topics middle school students have learned."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = QuizResponse,
    )
    return response.choices[0].message.parsed

In [33]:
response = completion(prompt)

In [34]:
response

QuizResponse(goal=['자기소개와 출신지에 대해 말할 수 있어요.', '자기소개와 출신지에 대해 대화할 때 적절한 표현을 사용할 수 있어요.'], preview_quizzes=[PreviewQuiz(fill_in_blank_quiz='A: My ___ is Felix. I’m from Australia. B: Glad to meet you.', korean_translation='A: 내 이름은 Felix야. 나는 Australia 출신이야. B: 만나서 반가워.', options=['age', 'name', 'friend'], answer='name', answer_index=2), PreviewQuiz(fill_in_blank_quiz='A: Hello, my name is Felix. ___ Australia. B: Great to meet you.', korean_translation='A: 안녕, 내 이름은 Felix야. ___ Australia 출신이야. B: 만나서 반가워.', options=['I’m from', 'am at', 'come from'], answer='I’m from', answer_index=1)], sample_script=[SampleScript(sample_script_eng='A: My name is Alex. I’m from Canada. B: Pleased to meet you. A: Hi, I am Alex. I come from Canada. B: Wonderful to make your acquaintance.', sample_script_kor='A: 내 이름은 Alex야. 나는 Canada 출신이야. B: 만나서 반가워. A: 안녕, 나는 Alex야. Canada에서 왔어. B: 반가워.')])

In [31]:
PreviewQuiz

__main__.PreviewQuiz

In [39]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent = 4)
print(response_output)

{
    "goal": [
        "자기소개와 출신지에 대해 말할 수 있어요.",
        "자기소개와 출신지에 대해 대화할 때 적절한 표현을 사용할 수 있어요."
    ],
    "preview_quizzes": [
        {
            "fill_in_blank_quiz": "A: My ___ is Felix. I’m from Australia. B: Glad to meet you.",
            "korean_translation": "A: 내 이름은 Felix야. 나는 Australia 출신이야. B: 만나서 반가워.",
            "options": [
                "age",
                "name",
                "friend"
            ],
            "answer": "name",
            "answer_index": 2
        },
        {
            "fill_in_blank_quiz": "A: Hello, my name is Felix. ___ Australia. B: Great to meet you.",
            "korean_translation": "A: 안녕, 내 이름은 Felix야. ___ Australia 출신이야. B: 만나서 반가워.",
            "options": [
                "I’m from",
                "am at",
                "come from"
            ],
            "answer": "I’m from",
            "answer_index": 1
        }
    ],
    "sample_script": [
        {
            "sample_script_eng": "A: My name is Alex. I

In [40]:
print(response)

goal=['자기소개와 출신지에 대해 말할 수 있어요.', '자기소개와 출신지에 대해 대화할 때 적절한 표현을 사용할 수 있어요.'] preview_quizzes=[PreviewQuiz(fill_in_blank_quiz='A: My ___ is Felix. I’m from Australia. B: Glad to meet you.', korean_translation='A: 내 이름은 Felix야. 나는 Australia 출신이야. B: 만나서 반가워.', options=['age', 'name', 'friend'], answer='name', answer_index=2), PreviewQuiz(fill_in_blank_quiz='A: Hello, my name is Felix. ___ Australia. B: Great to meet you.', korean_translation='A: 안녕, 내 이름은 Felix야. ___ Australia 출신이야. B: 만나서 반가워.', options=['I’m from', 'am at', 'come from'], answer='I’m from', answer_index=1)] sample_script=[SampleScript(sample_script_eng='A: My name is Alex. I’m from Canada. B: Pleased to meet you. A: Hi, I am Alex. I come from Canada. B: Wonderful to make your acquaintance.', sample_script_kor='A: 내 이름은 Alex야. 나는 Canada 출신이야. B: 만나서 반가워. A: 안녕, 나는 Alex야. Canada에서 왔어. B: 반가워.')]


In [ ]:
# type(response_output)

In [ ]:
# response.dict()

In [ ]:
# a = response.dict()['preview_quizzes']
# print(a[0]['fill_in_blank_quiz'])

# # for quiz in response.dict():
# #     a = quiz[]

In [ ]:
# for quiz in response.dict():
#     fill_in_blank_quiz = quiz.get('preview_quizzes',[])
#     print(fill_in_blank_quiz)

In [ ]:
# quizzes  = response.dict().get('preview_quizzes',[])
# fill_in_blanks = [quiz.get('fill_in_blank_quiz') for quiz in quizzes]

# for i, quiz in enumerate(fill_in_blanks):
#     print(f'Qioz {i} : {quiz}')

In [ ]:
# fill_in_blank_quiz = response.dict().get('preview_quizzes')
# print(type(fill_in_blank_quiz))
# print(fill_in_blank_quiz)
# # a = fill_in_blank_quiz.dict().get('fill_in_blank_quiz')
# # print(a)

# for문 돌려서 Batch API jsonl 파일 형식 만들기

확인 사항
- A1, A2, B1, B2, C1, C2 CEFR level 별로 퀴즈만 만들어지면 되는건가?
- 여섯개의 레벨 모두 만들어야하는지, 일부 레벨(A1, B1, C1)만 만들면 되는지도 결정

1. 커리큘럼 정보 csv 파일 읽어오기
2. for문으로 한줄씩 순회하면서 프롬프트 만들어주기들어주기
3. 렌더링한 프롬프트 파일에 써주기 (처음엔 조금만, ex. 5개 커리큘럼)
4. openai batch API 요청해서 task_id 받아오기 
5. batch task 완료되면 결과 받아오고 검수하기

In [41]:
def QuizMake(data, output_filename):
    jsonl_data = []

    for i in range(len(data)):
        prompt = prompt_template.render(
            description = data.loc[i,"DESCRIPTION"],
            key_expression = data.loc[i,"KEY_EXPRESSION"],
            sub_expression = [
                data.loc[i,"SUB_EXPRESSION"]
                # sample_data.loc[i,"SUB_EXPRESSION"].split("\n")[0],
                # sample_data.loc[i,"SUB_EXPRESSION"].split("\n")[1]
            ]
        )

        quiz_request = {
            "custom_id" : f"request-{i+1}",
            "method" : "POST",
            "url" : "/v1/chat/completions",
            "body" : {
                "model" : "o3-mini-2025-01-31",
                "messages" : [
                    {"role": "system", "content": "You will provide quizzes related to the topics students have learned, appropriately matching their CEFR level for middle school students."},
                    {"role": "user", "content": prompt}
                ],
                "response_format" : response_format
            }
        }

        jsonl_data.append(quiz_request)

        with open(output_filename, 'w', encoding='utf-8') as jsonl_file:
            for item in jsonl_data:
                jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f'JSONL 파일 생성 완료 : {output_filename}-{i+1}')

## 중학교 jsonl 파일

In [42]:
import pandas as pd

raw_data = pd.read_csv("../../DB/Extract_Mid.csv")
data_Mid1 = raw_data.loc[raw_data['학년'] == '중1']
data_Mid2 = raw_data.loc[raw_data['학년'] == '중2']
data_Mid3 = raw_data.loc[raw_data['학년'] == '중3']

# index 초기화화
data_Mid1 = data_Mid1.reset_index(drop=True)
data_Mid2 = data_Mid2.reset_index(drop=True)
data_Mid3 = data_Mid3.reset_index(drop=True)

display(data_Mid1.head(3))
display(data_Mid2.head(3))
display(data_Mid3.head(3))

,Unnamed: 0,textbook ID,학년,출판사,교육과정,Lesson,Title,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,0,701,중1,능률(김),2015,1,Welcome to My World,"자기소개와 출신지에 대해 말하고, 이에 반응하기.",A: My name is Felix. I’m from Australia. B: Gl...,A: 내 이름은 필릭스야. 나는 호주에서 왔어. B: 만나서 반가워.,"A: Hi, I’m (). I come from (). B: Nice to meet..."
1,1,701,중1,능률(김),2015,2,Discover Your Culture,be going to ()를 활용해 계획이나 예정된 일에 대해 묻고 답하기.,A: What are you going to do on your holiday? B...,A: 너는 휴가 때 무엇을 할 예정이야? B: 나는 여행을 갈 예정이야.,A: What are your plans for ()? B: I’m planning...
2,2,701,중1,능률(김),2015,3,"Spend Smart, Save Smart",You should ()를 활용해 조언하기.,"A: You should study harder. B: Okay, I’ll try.","A: 너는 더 열심히 공부해야 해. B: 알겠어, 노력할게.",A: It’s a good idea to (). B: I’ll give it a s...


,Unnamed: 0,textbook ID,학년,출판사,교육과정,Lesson,Title,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,7,801,중2,능률(김),2015,1,Express Yourself,be interested in을 활용해 관심사 묻고 답하기.,A: What are you interested in? B: I’m interest...,A: 너는 무엇에 관심이 있어? B: 나는 사진 촬영에 관심이 있어.,A: What do you like? B: I like (). A: What is ...
1,8,801,중2,능률(김),2015,2,"Eat Right, Be Happy",'d better ()를 활용해 충고하기.,You’d better take an umbrella.,너는 우산을 챙기는 게 좋겠어.,You should (). It’s a good idea to ().
2,9,801,중2,능률(김),2015,3,Understand the World,현재완료 시제 have ()를 활용해 경험 묻고 답하기.,"A: Have you ever traveled abroad? B: Yes, I ha...","A: 너 해외여행 해본 적 있어? B: 응, 해봤어. / 아니, 안 해봤어.","A: Have you tried ()? B: Yes, I have. / No, I ..."


,Unnamed: 0,textbook ID,학년,출판사,교육과정,Lesson,Title,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,14,901,중3,능률(김),2015,1,A Life Full of Experiences,What would you like to do?를 활용해 바람에 대해 묻고 답하기.,A: What would you like to do? B: I’d like to t...,A: 너는 무엇을 하고 싶어? B: 나는 세계를 여행하고 싶어.,A: What do you hope to do? B: I hope to (). A:...
1,15,901,중3,능률(김),2015,2,Take Care of Yourself,현재완료 have ()를 활용해 알고 있는지 묻기.,"A: Have you heard the latest news? B: Yes, I h...","A: 최신 소식 들었어? B: 응, 들었어. / 아니, 못 들었어.","A: Do you know about ()? B: Yes, I do. / No, I..."
2,16,901,중3,능률(김),2015,3,"Always Aware, Always Prepared",curious about ()를 활용해 궁금증 표현하기.,I'm curious about space exploration.,나는 우주 탐사에 대해 궁금해.,I wonder about (). I’d like to know about ().


In [ ]:
# import re

# split_data = ""

# for i in range(len(sample_data)):
#     text = sample_data.loc[i,'SUB_EXPRESSION']
#     text = str(text)
#     split_sentence = re.split(r'(?<=\.)\s(?=A:)', text)
#     sample_data.loc[i, "SUB_EXPRESSION"] = '"{}"'.format('"\n"'.join(split_sentence))

In [ ]:
# sample_data['SUB_EXPRESSION'][0]

In [ ]:
# sample_data.loc[0,"SUB_EXPRESSION"].split("\n")[1]

In [43]:
QuizMake(data_Mid1, output_filename = 'PreviewQuiz_Mid1_batch.jsonl')
QuizMake(data_Mid2, output_filename = 'PreviewQuiz_Mid2_batch.jsonl')
QuizMake(data_Mid3, output_filename = 'PreviewQuiz_Mid3_batch.jsonl')

JSONL 파일 생성 완료 : PreviewQuiz_Mid1_batch.jsonl-208
JSONL 파일 생성 완료 : PreviewQuiz_Mid2_batch.jsonl-144
JSONL 파일 생성 완료 : PreviewQuiz_Mid3_batch.jsonl-142


## 초등학교 jsonl 파일

In [44]:
import pandas as pd

raw_data = pd.read_csv("../../DB/Extract_Elem.csv")
data_Elem3 = raw_data.loc[raw_data['학년'] == '초3']
data_Elem4 = raw_data.loc[raw_data['학년'] == '초4']
data_Elem5 = raw_data.loc[raw_data['학년'] == '초5']
data_Elem6 = raw_data.loc[raw_data['학년'] == '초6']

# index 초기화화
data_Elem3 = data_Elem3.reset_index(drop=True)
data_Elem4 = data_Elem4.reset_index(drop=True)
data_Elem5 = data_Elem5.reset_index(drop=True)
data_Elem6 = data_Elem6.reset_index(drop=True)

display(data_Elem3.head(3))
display(data_Elem4.head(3))
display(data_Elem5.head(3))
display(data_Elem6.head(3))

,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,0,310,초3,YBM(김),2022,1,Hello! I’m Dito,Hello/Hi! 등의 표현을 사용해 만날 때 인사하기.,Hi there!,NaN,Hello! Hey!
1,1,310,초3,YBM(김),2022,2,"Sit Down, Please",극존칭 표현을 통해 요청하고 그에 긍정적으로 응답하기.,"A: Sit down, please. B: Sure.",NaN,A: Please take a seat. B: Certainly. A: Kindly...
2,2,310,초3,YBM(김),2022,3,What’s This?,의문문을 활용해 무슨 물건인지 묻고 답하기.,A: What's this? B: It's a box.,NaN,A: What's that? B: It's a (). A: What is this ...


,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,46,410,초4,YBM(김),2022,1,How Are You?,상대방의 안부를 물어보고 응답하기.,A: How are you doing? B: I'm doing fine.,NaN,A: How have you been? B: I'm feeling (). A: Is...
1,47,410,초4,YBM(김),2022,2,Let’s Play Baseball,Let's를 활용해 제안하고 이에 대해 답하기.,Let's go to the park.,NaN,Let's () together. How about we ()?
2,48,410,초4,YBM(김),2022,3,Where Is My Bag?,위치 묻기 표현을 활용하여 물건의 위치 묻고 답하기.,A: Where is my backpack? B: It's on the table.,NaN,A: Where is my pencil case? B: It's in the dra...


,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,92,503,초5,YBM(김),2015,1,How’s It Going?,How’s it going?을 활용해 안부 묻고 답하기.,How are you doing? — I'm doing well.,NaN,How have you been? — I've been doing well. How...
1,93,503,초5,YBM(김),2015,2,I’m in the Kitchen,의문사 Where를 활용해 사람의 위치 묻고 답하기.,A: Where are you? B: I'm in the living room.,NaN,A: Where is he? B: He's in the () room. A: Whe...
2,94,503,초5,YBM(김),2015,3,Whose Balloon Is This?,Whose의 소유격 의문사를 활용해 물건의 주인이 누구인지 묻고 답하기.,A: Whose bag is this? B: It's Donna's.,NaN,A: To whom does this belong? B: It's ()'s. A: ...


,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,119,603,초6,YBM(김),2015,1,What Grade Are You In?,What grade are you in? — I’m in the sixth grad...,A: What grade are you in? B: I'm in third grade.,NaN,A: What year are you in? B: I am in () grade. ...
1,120,603,초6,YBM(김),2015,2,What Would You Like?,'Would you like ()?'을 활용해 음식에 대해 묻고 답하기.,A: What would you like? B: I'd like strawberry...,NaN,A: What do you want to eat? B: I'd like (). A:...
2,121,603,초6,YBM(김),2015,3,My Favorite Subject Is Science,선호하는 과목에 대해 묻고 답하기.,A: What’s your favorite school subject? B: My ...,NaN,A: What class do you enjoy the most? B: My fav...


In [45]:
QuizMake(data_Elem3, output_filename = 'PreviewQuiz_Elem3_batch.jsonl')
QuizMake(data_Elem4, output_filename = 'PreviewQuiz_Elem4_batch.jsonl')
QuizMake(data_Elem5, output_filename = 'PreviewQuiz_Elem5_batch.jsonl')
QuizMake(data_Elem6, output_filename = 'PreviewQuiz_Elem6_batch.jsonl')

JSONL 파일 생성 완료 : PreviewQuiz_Elem3_batch.jsonl-240
JSONL 파일 생성 완료 : PreviewQuiz_Elem4_batch.jsonl-231
JSONL 파일 생성 완료 : PreviewQuiz_Elem5_batch.jsonl-99
JSONL 파일 생성 완료 : PreviewQuiz_Elem6_batch.jsonl-100


## 고등학교 jsonl 파일

In [ ]:
import pandas as pd

raw_data = pd.read_csv("../../DB/Extract_High.csv")
raw_data.head()
raw_data['제목'].value_counts()
# raw_data['출판사'].value_counts()
# data_Mid1 = raw_data.loc[raw_data['학년'] == '고1']
# data_Mid2 = raw_data.loc[raw_data['학년'] == '2']
# data_Mid3 = raw_data.loc[raw_data['학년'] == '중3']

# # index 초기화화
# data_Mid1 = data_Mid1.reset_index(drop=True)
# data_Mid2 = data_Mid2.reset_index(drop=True)
# data_Mid3 = data_Mid3.reset_index(drop=True)

# display(data_Mid1.head(3))
# display(data_Mid2.head(3))
# display(data_Mid3.head(3))

제목
영어        132
영어Ⅰ        48
영어Ⅱ        36
영어 I       24
영어 II      24
공통영어 1     12
공통영어 2     12
영어1        10
영어 2       10
기본영어       10
공통영어 I      8
Name: count, dtype: int64